In [1]:
import time
from pathlib import Path
import cv2
import torch
import torch.backends.cudnn as cudnn
from numpy import random
from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import check_img_size, check_requirements, check_imshow, \
    non_max_suppression, apply_classifier, \
    scale_coords, xyxy2xywh, strip_optimizer, set_logging, increment_path
from utils.plots import plot_one_box
from utils.torch_utils import select_device, load_classifier, \
                                time_synchronized, TracedModel

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Load model
weights = 'yolov7.pt'
model = attempt_load(weights, map_location=device)  # load FP32 model
stride = int(model.stride.max())  # model stride

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block


In [4]:
def detect(source, img_size=640, conf_thres=0.25, save_img=False):
    dataset = LoadImages(source, img_size=img_size)

    # Get names and colors
    names = model.module.names if hasattr(model, 'module') else model.names
    colors = [[random.randint(0, 255) for _ in range(3)] for _ in names]

    # Run inference
    if device.type != 'cpu':
        model(torch.zeros(1, 3, img_size, img_size).to(device).type_as(
            next(model.parameters())))  # run once
    old_img_w = old_img_h = img_size
    old_img_b = 1

    t0 = time.time()
    for path, img, im0s, vid_cap in dataset:
        img = torch.from_numpy(img).to(device)
        img = img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        # Inference
        t1 = time_synchronized()
        pred = model(img)[0]
        t2 = time_synchronized()

        # Apply NMS
        pred = non_max_suppression(pred, conf_thres)
        t3 = time_synchronized()

        # Process detections
        for i, det in enumerate(pred):  # detections per image
            p, s, im0, frame = path, '', im0s, getattr(dataset, 'frame', 0)

            p = Path(p)  # to Path
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
            if len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

                # Print results
                for c in det[:, -1].unique():
                    n = (det[:, -1] == c).sum()  # detections per class
                    s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string

                # Write results
                for *xyxy, conf, cls in reversed(det):
                    # Add bbox to image
                    label = f'{names[int(cls)]} {conf:.2f}'
                    plot_one_box(xyxy, im0, label=label, color=colors[int(cls)], line_thickness=1)
                    print(label)

            cv2.imshow(str(p), im0)
            if source.endswith('mp4'):
                cv2.waitKey(1)
            else:
                cv2.waitKey(0)
                cv2.destroyAllWindows()
                
    if source.endswith('mp4'):
        cv2.destroyAllWindows()
        
    # Print time (inference + NMS)
    print(f'{s}Done. ({(1E3 * (t2 - t1)):.1f}ms) Inference, ({(1E3 * (t3 - t2)):.1f}ms) NMS')
    print(f'Done. ({time.time() - t0:.3f}s)')
    return

In [6]:
# detect('./inference/images/horses.jpg')
detect('./test.mp4')

C:\anaconda3\lib\site-packages\torch\functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


video 1/1 (1/443) F:\0_AI\Books\PyTorch\yolov7\test.mp4: person 0.26
handbag 0.31
handbag 0.35
person 0.54
person 0.56
person 0.60
person 0.66
bicycle 0.67
potted plant 0.67
person 0.68
handbag 0.69
person 0.72
person 0.74
person 0.79
bicycle 0.79
person 0.80
bicycle 0.80
person 0.86
person 0.87
person 0.88
person 0.88
person 0.88
person 0.89
person 0.91
person 0.92
video 1/1 (2/443) F:\0_AI\Books\PyTorch\yolov7\test.mp4: person 0.26
bench 0.28
handbag 0.35
person 0.51
handbag 0.55
person 0.57
person 0.58
person 0.65
bicycle 0.67
potted plant 0.67
person 0.69
handbag 0.70
person 0.73
person 0.77
person 0.79
bicycle 0.80
person 0.80
bicycle 0.80
person 0.82
person 0.87
person 0.87
person 0.88
person 0.89
person 0.89
person 0.91
person 0.92
video 1/1 (3/443) F:\0_AI\Books\PyTorch\yolov7\test.mp4: person 0.27
bench 0.28
handbag 0.35
person 0.51
handbag 0.53
person 0.57
person 0.59
person 0.65
bicycle 0.67
potted plant 0.68
person 0.69
handbag 0.70
person 0.73
person 0.77
person 0.79
bicyc

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 4.00 GiB total capacity; 3.36 GiB already allocated; 0 bytes free; 3.49 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF